In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold,cross_validate
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler


from sklearn.pipeline import FeatureUnion, make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


In [2]:
TRAIN_PATH = 'data/train.csv'
TEST_PATH = 'data/test.csv'
RANDOM_STATE=100

In [3]:
df_train = pd.read_csv(TRAIN_PATH, index_col=0)

In [4]:
X = df_train.drop('target', axis=1)
y = df_train['target']
del df_train

In [5]:
X.head()

,id,vas_id,buy_time,0,1,2,3,4,5,6,...,243,244,245,246,247,248,249,250,251,252
0,540968,8.0,1537131600,-31.559971,327.360888,-45.500786,274.753202,-50.918246,253.64821,49.15382,...,-845.373846,-613.770792,-20.996269,-37.630448,-28.747724,4.167111,7.305572,-12.175933,21.54386,0.0
1,1454121,4.0,1531688400,547.270029,238.430888,533.329214,274.803202,527.911754,253.69821,-16.08618,...,-972.373846,-613.770792,-25.996269,-19.630448,-278.747724,-24.832889,-0.694428,-11.175933,-0.45614,0.0
2,2458816,1.0,1534107600,-92.139971,-95.469112,-106.080786,-139.596798,-111.498246,-142.90179,-16.08618,...,-977.373846,-613.770792,-25.996269,-37.630448,-304.747724,-25.832889,-0.694428,-12.175933,-0.45614,0.0
3,3535012,5.0,1535922000,54.880029,12.970888,54.079214,-9.116798,48.661754,-30.22179,60.17382,...,-977.373846,-613.770792,-25.996269,-18.630448,-133.747724,-14.832889,-0.694428,-1.175933,-0.45614,0.0
4,1693214,1.0,1535922000,45.160029,295.240888,64.679214,344.283202,59.261754,323.17821,-16.08618,...,-965.373846,-612.770792,-22.996269,-32.630448,-127.747724,-4.832889,-0.694428,-12.175933,-0.45614,0.0


In [6]:
X_nunique = X.apply(lambda x: x.nunique(dropna=False))
f_all = set(X_nunique.index.tolist())
print(f'Количество признаков: {len(f_all)}')

Количество признаков: 256


In [7]:
f_const = set(X_nunique[X_nunique == 1].index.tolist())
print(f'Количество константных признаков: {len(f_const)}')

Количество константных признаков: 5


In [8]:
f_numeric = (X.fillna(0).astype(int).sum() - X.fillna(0).sum()).abs()
f_numeric = set(f_numeric[f_numeric > 0].index.tolist())
print(f'Количество числовых признаков: {len(f_numeric)}')

Количество числовых признаков: 247


In [9]:
f_binary = set(X.loc[:, f_numeric].columns[(
                      (X.loc[:, f_numeric].max() == 1) & \
                      (X.loc[:, f_numeric].min() == 0) & \
                      (X.loc[:, f_numeric].isnull().sum() == 0))])
print(f'Количество Бинарных признаков: {len(f_binary)}')


Количество Бинарных признаков: 0


In [10]:
f_other = f_all - (f_numeric | f_const)
print(f'Количество остальных признаков: {len(f_other)}')

Количество остальных признаков: 4


In [11]:
f_time = {'buy_time'}

In [12]:
f_categorical = set(X_nunique.loc[f_numeric][X_nunique.loc[f_numeric] < 10].index.tolist())
f_categorical
print(f'Количество категориальных признаков: {len(f_categorical)}')

Количество категориальных признаков: 28


In [13]:
f_ok = list(f_categorical | f_numeric ) +['vas_id']
f_categorical, f_numeric = list(f_categorical)+['vas_id'], list(f_numeric)

Для теста возьмем 3 последних даты

In [14]:
train_idx = X[X['buy_time'] < 1544994000].index
test_idx = X[X['buy_time'] >= 1544994000].index

In [15]:
print(len(test_idx)/len(train_idx))
print(len(train_idx)+len(test_idx))

0.30350478752000354
831653


In [16]:
#train_idx, test_idx = train_test_split(X.index, test_size=0.20, random_state=RANDOM_STATE)

y_train, y_test = y.loc[train_idx], y.loc[test_idx]
X_train, X_test = X.loc[train_idx], X.loc[test_idx]

print("Train")
print(y.loc[train_idx].value_counts()/train_idx.shape[0])
print()
print("Test")
print(y.loc[test_idx].value_counts()/test_idx.shape[0])

Train
0.0    0.929307
1.0    0.070693
Name: target, dtype: float64

Test
0.0    0.922108
1.0    0.077892
Name: target, dtype: float64


In [17]:
disbalance = y_train.value_counts()[0] / y_train.value_counts()[1]
disbalance

13.145688756845443

In [18]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        assert isinstance(X, pd.DataFrame)

        try:
            return X[self.columns]
        except KeyError:
            cols_error = list(set(self.columns) - set(X.columns))
            raise KeyError("DataFrame не содердит следующие колонки: %s" % cols_error)

In [19]:
f_prep_pipeline = make_pipeline(
    ColumnSelector(columns=f_ok),
    FeatureUnion(transformer_list=[
        ("numeric_features", make_pipeline(
            ColumnSelector(f_numeric),
            StandardScaler()
        )),
        ("categorical_features", make_pipeline(
            ColumnSelector(f_categorical),
            OneHotEncoder(handle_unknown='ignore')
        ))
        
    ])
)

In [20]:
from catboost import CatBoostClassifier

In [21]:
cb_pipe = make_pipeline(
    f_prep_pipeline,
    CatBoostClassifier(class_weights=[1, disbalance],
                       random_state=RANDOM_STATE)
)

In [22]:
kfold_cv = KFold(n_splits=3, shuffle=True, random_state=RANDOM_STATE)

def run_cv(estimator, cv, X, y, scoring='f1_macro', model_name=""):
    cv_res = cross_validate(estimator, X, y, cv=cv, scoring=scoring, n_jobs=-1)
    
    print("%s: %s = %0.2f (+/- %0.2f)" % (model_name,
                                         scoring,
                                         cv_res['test_score'].mean(),
                                         cv_res['test_score'].std() * 2))
def run_grid_search(estimator, X, y, params_grid, cv, scoring='f1_macro'):
    gsc = GridSearchCV(estimator, params_grid, scoring=scoring, cv=cv)

    gsc.fit(X, y)
    print("Best %s score: %.2f" % (scoring, gsc.best_score_))
    print()
    print("Best parameters set found on development set:")
    print()
    print(gsc.best_params_)
    print()
    print("Grid scores on development set:")
    print()

    for i, params in enumerate(gsc.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (gsc.cv_results_['mean_test_score'][i], gsc.cv_results_['std_test_score'][i] * 2, params))

    print()
    
    return gsc

In [23]:
[k for k in cb_pipe.get_params().keys() if 'class' in k]

['catboostclassifier',
 'catboostclassifier__class_weights',
 'catboostclassifier__random_state']

In [24]:
# param_grid = {
#     "catboostclassifier__class_weights": [1, disbalance],
#     "catboostclassifier__max_depth": [5, 7],
#     "catboostclassifier__learning_rate":[0.01],
#     "catboostclassifier__iterations": [100, 200]
    
# }

# cb_fs_gsc = run_grid_search(cb_pipe, X_train, y_train, param_grid, kfold_cv)

In [25]:
final_model = make_pipeline(
    f_prep_pipeline,
    CatBoostClassifier(class_weights=[1, disbalance],
                       learning_rate=0.01,
                       max_depth=5,
                       eval_metric='F1',
                       random_state=RANDOM_STATE)
)

final_model.fit(X_train, y_train)

0:	learn: 0.7767714	total: 542ms	remaining: 9m 1s
1:	learn: 0.7767707	total: 837ms	remaining: 6m 57s
2:	learn: 0.7767707	total: 1.14s	remaining: 6m 18s
3:	learn: 0.7767707	total: 1.44s	remaining: 5m 59s
4:	learn: 0.7767707	total: 1.72s	remaining: 5m 43s
5:	learn: 0.7767707	total: 2.09s	remaining: 5m 45s
6:	learn: 0.7767707	total: 2.4s	remaining: 5m 39s
7:	learn: 0.7767707	total: 2.69s	remaining: 5m 33s
8:	learn: 0.7767707	total: 2.97s	remaining: 5m 27s
9:	learn: 0.7767707	total: 3.32s	remaining: 5m 29s
10:	learn: 0.7767707	total: 3.67s	remaining: 5m 30s
11:	learn: 0.7767707	total: 3.97s	remaining: 5m 26s
12:	learn: 0.7767707	total: 4.29s	remaining: 5m 25s
13:	learn: 0.7767707	total: 4.62s	remaining: 5m 25s
14:	learn: 0.7767707	total: 4.96s	remaining: 5m 25s
15:	learn: 0.7767707	total: 5.25s	remaining: 5m 22s
16:	learn: 0.7767707	total: 5.57s	remaining: 5m 21s
17:	learn: 0.7767707	total: 5.85s	remaining: 5m 19s
18:	learn: 0.7767707	total: 6.16s	remaining: 5m 17s
19:	learn: 0.7767707	tot

158:	learn: 0.7767707	total: 56.7s	remaining: 5m
159:	learn: 0.7767707	total: 57.2s	remaining: 5m
160:	learn: 0.7767707	total: 57.6s	remaining: 4m 59s
161:	learn: 0.7767707	total: 58s	remaining: 4m 59s
162:	learn: 0.7767707	total: 58.4s	remaining: 4m 59s
163:	learn: 0.7767707	total: 58.8s	remaining: 4m 59s
164:	learn: 0.7767707	total: 59.3s	remaining: 4m 59s
165:	learn: 0.7767707	total: 59.8s	remaining: 5m
166:	learn: 0.7767707	total: 1m	remaining: 5m
167:	learn: 0.7767707	total: 1m	remaining: 5m
168:	learn: 0.7767707	total: 1m 1s	remaining: 5m
169:	learn: 0.7767707	total: 1m 1s	remaining: 5m
170:	learn: 0.7767707	total: 1m 1s	remaining: 5m
171:	learn: 0.7767707	total: 1m 2s	remaining: 5m
172:	learn: 0.7767707	total: 1m 2s	remaining: 5m
173:	learn: 0.7767707	total: 1m 3s	remaining: 5m
174:	learn: 0.7767707	total: 1m 3s	remaining: 4m 59s
175:	learn: 0.7767707	total: 1m 4s	remaining: 4m 59s
176:	learn: 0.7767707	total: 1m 4s	remaining: 4m 59s
177:	learn: 0.7767707	total: 1m 4s	remaining:

313:	learn: 0.7767707	total: 2m 5s	remaining: 4m 33s
314:	learn: 0.7767707	total: 2m 5s	remaining: 4m 32s
315:	learn: 0.7767707	total: 2m 5s	remaining: 4m 32s
316:	learn: 0.7767707	total: 2m 6s	remaining: 4m 32s
317:	learn: 0.7767707	total: 2m 6s	remaining: 4m 31s
318:	learn: 0.7767707	total: 2m 6s	remaining: 4m 30s
319:	learn: 0.7767707	total: 2m 7s	remaining: 4m 30s
320:	learn: 0.7767707	total: 2m 7s	remaining: 4m 29s
321:	learn: 0.7767707	total: 2m 7s	remaining: 4m 29s
322:	learn: 0.7767707	total: 2m 8s	remaining: 4m 28s
323:	learn: 0.7767707	total: 2m 8s	remaining: 4m 28s
324:	learn: 0.7767707	total: 2m 8s	remaining: 4m 27s
325:	learn: 0.7767707	total: 2m 9s	remaining: 4m 27s
326:	learn: 0.7767707	total: 2m 9s	remaining: 4m 26s
327:	learn: 0.7767707	total: 2m 9s	remaining: 4m 25s
328:	learn: 0.7767707	total: 2m 10s	remaining: 4m 25s
329:	learn: 0.7767707	total: 2m 10s	remaining: 4m 24s
330:	learn: 0.7767707	total: 2m 10s	remaining: 4m 24s
331:	learn: 0.7767707	total: 2m 11s	remaini

466:	learn: 0.7767707	total: 3m 2s	remaining: 3m 27s
467:	learn: 0.7767707	total: 3m 2s	remaining: 3m 27s
468:	learn: 0.7767707	total: 3m 2s	remaining: 3m 27s
469:	learn: 0.7767707	total: 3m 3s	remaining: 3m 26s
470:	learn: 0.7767707	total: 3m 3s	remaining: 3m 26s
471:	learn: 0.7767707	total: 3m 4s	remaining: 3m 26s
472:	learn: 0.7767707	total: 3m 4s	remaining: 3m 25s
473:	learn: 0.7767707	total: 3m 5s	remaining: 3m 25s
474:	learn: 0.7767707	total: 3m 5s	remaining: 3m 25s
475:	learn: 0.7767707	total: 3m 6s	remaining: 3m 24s
476:	learn: 0.7767707	total: 3m 6s	remaining: 3m 24s
477:	learn: 0.7767707	total: 3m 6s	remaining: 3m 24s
478:	learn: 0.7767707	total: 3m 7s	remaining: 3m 23s
479:	learn: 0.7767707	total: 3m 7s	remaining: 3m 23s
480:	learn: 0.7767707	total: 3m 8s	remaining: 3m 23s
481:	learn: 0.7767707	total: 3m 8s	remaining: 3m 22s
482:	learn: 0.7767707	total: 3m 9s	remaining: 3m 22s
483:	learn: 0.7767707	total: 3m 9s	remaining: 3m 22s
484:	learn: 0.7767707	total: 3m 10s	remaining:

620:	learn: 0.7767707	total: 4m 12s	remaining: 2m 33s
621:	learn: 0.7767707	total: 4m 12s	remaining: 2m 33s
622:	learn: 0.7767707	total: 4m 12s	remaining: 2m 32s
623:	learn: 0.7767707	total: 4m 13s	remaining: 2m 32s
624:	learn: 0.7767707	total: 4m 13s	remaining: 2m 31s
625:	learn: 0.7767707	total: 4m 13s	remaining: 2m 31s
626:	learn: 0.7767707	total: 4m 13s	remaining: 2m 31s
627:	learn: 0.7767707	total: 4m 14s	remaining: 2m 30s
628:	learn: 0.7767707	total: 4m 14s	remaining: 2m 30s
629:	learn: 0.7767707	total: 4m 14s	remaining: 2m 29s
630:	learn: 0.7767707	total: 4m 15s	remaining: 2m 29s
631:	learn: 0.7767707	total: 4m 15s	remaining: 2m 28s
632:	learn: 0.7767707	total: 4m 15s	remaining: 2m 28s
633:	learn: 0.7767707	total: 4m 15s	remaining: 2m 27s
634:	learn: 0.7767707	total: 4m 16s	remaining: 2m 27s
635:	learn: 0.7767707	total: 4m 16s	remaining: 2m 26s
636:	learn: 0.7767707	total: 4m 16s	remaining: 2m 26s
637:	learn: 0.7767707	total: 4m 17s	remaining: 2m 25s
638:	learn: 0.7767707	total:

773:	learn: 0.7767707	total: 5m 9s	remaining: 1m 30s
774:	learn: 0.7767707	total: 5m 10s	remaining: 1m 30s
775:	learn: 0.7767707	total: 5m 10s	remaining: 1m 29s
776:	learn: 0.7767707	total: 5m 10s	remaining: 1m 29s
777:	learn: 0.7767707	total: 5m 11s	remaining: 1m 28s
778:	learn: 0.7767707	total: 5m 11s	remaining: 1m 28s
779:	learn: 0.7767707	total: 5m 12s	remaining: 1m 28s
780:	learn: 0.7767707	total: 5m 12s	remaining: 1m 27s
781:	learn: 0.7767707	total: 5m 12s	remaining: 1m 27s
782:	learn: 0.7767707	total: 5m 13s	remaining: 1m 26s
783:	learn: 0.7767707	total: 5m 13s	remaining: 1m 26s
784:	learn: 0.7767707	total: 5m 13s	remaining: 1m 25s
785:	learn: 0.7767707	total: 5m 14s	remaining: 1m 25s
786:	learn: 0.7767707	total: 5m 15s	remaining: 1m 25s
787:	learn: 0.7767707	total: 5m 15s	remaining: 1m 24s
788:	learn: 0.7767707	total: 5m 15s	remaining: 1m 24s
789:	learn: 0.7767707	total: 5m 16s	remaining: 1m 24s
790:	learn: 0.7767707	total: 5m 17s	remaining: 1m 23s
791:	learn: 0.7767707	total: 

928:	learn: 0.7767707	total: 6m 17s	remaining: 28.8s
929:	learn: 0.7767707	total: 6m 17s	remaining: 28.4s
930:	learn: 0.7767707	total: 6m 17s	remaining: 28s
931:	learn: 0.7767707	total: 6m 18s	remaining: 27.6s
932:	learn: 0.7767707	total: 6m 18s	remaining: 27.2s
933:	learn: 0.7767707	total: 6m 18s	remaining: 26.8s
934:	learn: 0.7767707	total: 6m 19s	remaining: 26.4s
935:	learn: 0.7767707	total: 6m 19s	remaining: 25.9s
936:	learn: 0.7767707	total: 6m 19s	remaining: 25.5s
937:	learn: 0.7767707	total: 6m 20s	remaining: 25.1s
938:	learn: 0.7767707	total: 6m 20s	remaining: 24.7s
939:	learn: 0.7767707	total: 6m 20s	remaining: 24.3s
940:	learn: 0.7767707	total: 6m 20s	remaining: 23.9s
941:	learn: 0.7767707	total: 6m 21s	remaining: 23.5s
942:	learn: 0.7767707	total: 6m 21s	remaining: 23.1s
943:	learn: 0.7767707	total: 6m 21s	remaining: 22.7s
944:	learn: 0.7767707	total: 6m 22s	remaining: 22.3s
945:	learn: 0.7767707	total: 6m 22s	remaining: 21.8s
946:	learn: 0.7767707	total: 6m 22s	remaining: 2

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columnselector',
                                  ColumnSelector(columns=['98', '88', '58',
                                                          '82', '189', '224',
                                                          '51', '0', '86',
                                                          '242', '176', '166',
                                                          '132', '160', '232',
                                                          '221', '140', '163',
                                                          '45', '12', '25',
                                                          '175', '201', '37',
                                                          '6', '142', '246',
                                                          '2', '56', '169', ...])),
                                 ('featureunion',
                                  FeatureUnion(transformer_list=[('numeric_features',
  

In [26]:
def evaluate_model(model, X_test, y_test):
    y_test_pred = model.predict_proba(X_test)[:,1]
    print(f'{classification_report(y_test, y_test_pred > 0.5)}\n\n')

In [27]:
evaluate_model(final_model, X_test, y_test)

              precision    recall  f1-score   support

         0.0       1.00      0.87      0.93    178557
         1.0       0.39      1.00      0.56     15083

    accuracy                           0.88    193640
   macro avg       0.69      0.93      0.74    193640
weighted avg       0.95      0.88      0.90    193640





In [29]:
import pickle

In [30]:
with open('model.pkl', 'wb') as f:
    pickle.dump(final_model, f)